In [6]:
from __future__ import print_function
import datetime
print(datetime.datetime.now())
import http.server
import urllib.parse
from urllib.parse import urlparse
import matplotlib
import datetime
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

import time
import pyodbc
import sys
from shutil import copyfile

import concurrent.futures

from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.recurrent import LSTM, GRU
from keras.optimizers import RMSprop, Adam, SGD
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.preprocessing import MultiLabelBinarizer
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers import TimeDistributed
from keras.callbacks import EarlyStopping

import gc

import tensorflow as tf

from keras import backend as K
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))
    
# Set Epochs
isFast = False

# we need around 2000 epochs to see good accuracy. For testing 100 epochs will do.
EPOCHS = 10 if isFast else 100
BATCH_SIZE_FOR_TRAIN = 50
VERBOSE_MODE=0
NUM_OUTPUT_CLASSES=3

# Specify the internal-state dimensions of the LSTM cell
#zipcode, city, state
H_DIMS =  3
T_LAG = 1
NUM_FEATURES_PER_SAMPLE = (H_DIMS * T_LAG) #KERAS

PROPERTY_APPROVAL=['ACCEPT','REJECT','GRAY']

print('OK')





2020-03-27 04:54:41.037069
OK


In [ ]:
#Read the CSV Data first
histData = pd.read_csv("C:\\Users\\ATOM_PROD1\\OneDrive\\LADDU\\DS4A-LATAM\\Project-CREDITAS-PropertyApproval-2020\\MVP\\PreviousDecisions-v1.csv")
histData.head()


In [93]:
histData.count()

Unnamed: 0          41770
model_decision      41761
analyst_decision    39742
zip_code            41768
concat              41770
coordinates         41757
dtype: int64

In [94]:
histData['model_decision'].fillna('NA',inplace=True)
histData['analyst_decision'].fillna('NA',inplace=True)
histData['zip_code'].fillna('000000',inplace=True)
histData['coordinates'].fillna('(0,0)',inplace=True)

histData.count()

Unnamed: 0          41770
model_decision      41770
analyst_decision    41770
zip_code            41770
concat              41770
coordinates         41770
dtype: int64

In [95]:
print('model_decision values:')
pd.Index(histData['model_decision']).value_counts()

model_decision values:


G        38001
error     1629
A         1247
R          884
NA           9
Name: model_decision, dtype: int64

In [96]:
print('analyst_decision values:')
pd.Index(histData['analyst_decision']).value_counts()

analyst_decision values:


R     22653
A     17089
NA     2028
Name: analyst_decision, dtype: int64

In [97]:
print('unique values:')
pd.Index(histData['model_decision']+'__'+histData['analyst_decision']).value_counts()

unique values:


G__R         21648
G__A         16282
A__NA         1247
error__R       829
error__A       795
R__NA          703
R__R           169
G__NA           71
R__A            12
NA__R            7
error__NA        5
NA__NA           2
dtype: int64

In [100]:
same_count = 0
diff_count = 0

accept_count = 0
reject_count = 0
undecided_count = 0

accept_states_list = []
accept_states_moreinfo = []

reject_states_list = []
reject_states_moreinfo = []

#print(histData['concat'].rsplit('/', 1)[-1])

for index, row in histData.iterrows():
    if row['model_decision'] != row['analyst_decision']:
        diff_count = diff_count + 1     
    else:
        same_count = same_count + 1
    if row['model_decision'] == 'A' or row['analyst_decision'] == 'A':
        accept_count = accept_count + 1
        accept_states_list.append(row['concat'].rsplit(',', 1)[-1])
        accept_states_moreinfo.append(row['concat'].rsplit(',')[2:])
        
    elif row['model_decision'] == 'R' or row['analyst_decision'] == 'R':
        reject_count = reject_count + 1
        reject_states_list.append(row['concat'].rsplit(',', 1)[-1])
        reject_states_moreinfo.append(row['concat'].rsplit(',')[2:])
    else:
        undecided_count = undecided_count + 1
        
print("Total Count:%d, Diff_count:%d, Same_Count:%d" %((same_count+diff_count),diff_count, same_count))
print("Accept:%d(%d), Reject:%d, undecided:%d" %(accept_count,((accept_count/(same_count+diff_count))*100),reject_count,undecided_count))


Total Count:41770, Diff_count:41599, Same_Count:171
Accept:18336(43), Reject:23356, undecided:78


In [101]:
accept_val_list = pd.Index(accept_states_list).value_counts()
reject_val_list = pd.Index(reject_states_list).value_counts()


In [102]:
for keyVal in accept_val_list.keys():
    if(keyVal != ' '):
        totalVal = (accept_val_list[keyVal] + reject_val_list[keyVal])
        print('State:%s A:%d(%d), R:%d' %(keyVal, accept_val_list[keyVal],((accept_val_list[keyVal]/totalVal)*100), reject_val_list[keyVal]))


State: SP A:10658(48), R:11492
State: PR A:1549(52), R:1421
State: MG A:1268(42), R:1724
State: RJ A:1007(33), R:1966
State: SC A:962(44), R:1223
State: RS A:853(41), R:1208
State: GO A:261(31), R:572
State: DF A:251(38), R:399
State: MS A:231(39), R:349
State: BA A:212(29), R:514
State: CE A:188(39), R:283
State: PE A:156(28), R:391
State: ES A:148(28), R:376
State: PB A:128(45), R:156
State: MT A:108(27), R:279
State: RN A:85(37), R:141
State: SE A:55(36), R:95
State: PA A:50(24), R:152
State: AM A:40(13), R:250
State: PI A:40(40), R:59
State: MA A:36(17), R:169
State: AL A:33(26), R:92
State: TO A:16(28), R:41


In [126]:
new_accept_list = []
new_reject_list = []

for row in accept_states_moreinfo:
    new_accept_list.append(row[0] + ',' + row[1])

for row in reject_states_moreinfo:
    new_reject_list.append(row[0] + ',' + row[1])

accept_val_list = pd.Index(new_accept_list).value_counts()
reject_val_list = pd.Index(new_reject_list).value_counts()    

for keyVal in accept_val_list.keys():
    if(keyVal != ' '):
        if((keyVal in reject_val_list) == True):
            totalVal = (accept_val_list[keyVal] + reject_val_list[keyVal])
            print('State:%s A:%d(%d), R:%d' %(keyVal, accept_val_list[keyVal],((accept_val_list[keyVal]/totalVal)*100), reject_val_list[keyVal]))

    

State: São Paulo, SP A:3495(52), R:3191
State: Curitiba, PR A:689(62), R:422
State: Rio de Janeiro, RJ A:627(45), R:759
State: Campinas, SP A:391(53), R:340
State: Guarulhos, SP A:358(46), R:410
State: Belo Horizonte, MG A:349(43), R:451
State: Santo André, SP A:346(57), R:252
State: Porto Alegre, RS A:324(53), R:286
State: São Bernardo do Campo, SP A:288(55), R:228
State: Sorocaba, SP A:250(49), R:252
State: Brasília, DF A:229(37), R:386
State: São José dos Campos, SP A:226(51), R:213
State: Jundiaí, SP A:225(50), R:220
State: Praia Grande, SP A:215(69), R:95
State: Ribeirão Preto, SP A:210(52), R:190
State: Uberlândia, MG A:208(53), R:179
State: Goiânia, GO A:195(44), R:244
State: Campo Grande, MS A:187(40), R:273
State: Osasco, SP A:185(37), R:310
State: Santos, SP A:175(69), R:77
State: Fortaleza, CE A:173(40), R:254
State: Londrina, PR A:171(55), R:135
State: Florianópolis, SC A:162(55), R:129
State: Joinville, SC A:153(36), R:265
State: São José do Rio Preto, SP A:148(57), R:110
